In [1]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np

tqdm.pandas()

In [2]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar = True)

INFO: Pandarallel will run on 48 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
data = pd.read_parquet("standard_counts.parquet")

In [4]:
from collections import defaultdict
groups = defaultdict(list)

unique_indicies = []

In [5]:
for row in tqdm(data.itertuples(), total = len(data)):
    groups[row.Hash].append(row._1)

  0%|          | 0/146432000 [00:00<?, ?it/s]

In [6]:
removable_hashes = []
for group_hash in tqdm(groups.keys()):
    if len(groups[group_hash]) < 2:
        removable_hashes.append(group_hash)

  0%|          | 0/145907878 [00:00<?, ?it/s]

In [7]:
for group_hash in tqdm(removable_hashes):
    index = groups.pop(group_hash)[0]
    unique_indicies.append(index)

  0%|          | 0/145644401 [00:00<?, ?it/s]

In [10]:
from mmap_dataset import MMapIndexedDataset

dataset = MMapIndexedDataset('/scratch/pile/standard/document', skip_warmup = True)

def get_arr_from_idx(idx):
    return dataset[idx][:64]

    reading sizes...
    reading pointers...
    reading document index...
    creating numpy buffer of mmap...
    creating memory view of numpy buffer...


In [11]:
for indicies in tqdm(groups.values()):
    duplicate_indicies = set()
    for i in range(len(indicies)):
        if indicies[i] in duplicate_indicies:
            continue
        
        unique_indicies.append(indicies[i])
        for j in range(i+1, len(indicies)):
            arr1 = get_arr_from_idx(i)
            arr2 = get_arr_from_idx(j)
            if (arr1 == arr2).all():
                duplicate_indicies.add(indicies[j])
        

  0%|          | 0/263477 [00:00<?, ?it/s]